In [1]:
import pandas as pd
import numpy as np
sample_submission_df = pd.read_csv("./datasets/sample_submission_zingat.csv")
test_set = pd.read_csv("./datasets/zingat_usecase_testset_null.csv")
train_set = pd.read_csv("./datasets/zingat_usecase_trainset.csv")

## Variable Change

In [2]:
def change_variable(data):
    
    data = data.copy()
    return data.rename(columns = {
        'odasayısı': 'odasayisi',
        'brütm²': 'brutm2',
        'netm²': 'netm2',
        'binayaşı': 'binayasi',
        'bulunduğukat': 'bulundugukat',
        'binadakikatsayısı': 'binadakikatsayisi',
        'isıtmatipi': 'isitimatipi',
        'İnterkom': 'interkom',
        'depremyönetmeliğineuygun': 'depremyonetmeligineuygun',
        'asansör': 'asansor',
        'Çocukoyunalanı': 'cocukoyunalani',
        'giyinmeodası': 'giyinmeodasi',
        'güncel_fiyat': 'guncelfiyat',
    })

In [3]:
train_set = change_variable(train_set)

## Converting The Yes-None Values to 0-1

In [4]:
def convert_values(data):
    
    data.loc[data.interkom == "VAR", "interkom"] , data.loc[data.interkom == "YOK", "interkom"] = 1,0
    data.loc[data.depremyonetmeligineuygun == "VAR", "depremyonetmeligineuygun"] , data.loc[data.depremyonetmeligineuygun == "YOK", "depremyonetmeligineuygun"] = 1,0
    data.loc[data.asansor == "VAR", "asansor"] , data.loc[data.asansor == "YOK", "asansor"] = 1,0
    data.loc[data.cocukoyunalani == "VAR", "cocukoyunalani"] , data.loc[data.cocukoyunalani == "YOK", "cocukoyunalani"] = 1,0
    data.loc[data.giyinmeodasi == "VAR", "giyinmeodasi"] , data.loc[data.giyinmeodasi == "YOK", "giyinmeodasi"] = 1,0
    data.loc[data.ebeveynbanyosu == "VAR", "ebeveynbanyosu"] , data.loc[data.ebeveynbanyosu == "YOK", "ebeveynbanyosu"] = 1,0

In [5]:
convert_values(train_set)

## Convert Time Series

In [8]:
def convert_time_series(data):
    
    data.tarih = pd.DatetimeIndex(data.tarih)

In [9]:
convert_time_series(train_set)

In [10]:
train_set.tarih

0       2019-01-01
1       2019-01-01
2       2019-01-01
3       2019-01-01
4       2019-01-01
           ...    
64568   2020-04-19
64569   2020-04-19
64570   2020-04-19
64571   2020-04-19
64572   2020-04-19
Name: tarih, Length: 64573, dtype: datetime64[ns]

## Convert Price

In [6]:
def convert_price(data):
    
    data["guncelfiyat"] = data.guncelfiyat.str[:-3]
    data["guncelfiyat"] = data["guncelfiyat"].astype(str).astype(int)

In [7]:
convert_price(train_set)
train_set.guncelfiyat

0        1050000
1         547500
2         440000
3        1450000
4         169000
          ...   
64568     790000
64569     340000
64570     620000
64571     750000
64572     250000
Name: guncelfiyat, Length: 64573, dtype: int32

In [10]:
def tarih_fonksiyonu(data):
   
    data = data.copy()
    data['yil'], data['ay'], data['gun'] = data["tarih"].str.split("-",2)
    data["yil"] = data["yil"].astype(str).astype(int)
    data["ay"] = data["ay"].astype(str).astype(int)
    data["gun"] = data["gun"].astype(str).astype(int)
    
    return data

In [11]:
train_set = tarih_fonksiyonu(train_set)

ValueError: too many values to unpack (expected 3)

In [12]:
def belirtilmemis_fonksiyonu(data):
    
    data = data.copy()
    data = data.replace("-","belirtilmemis")
    
    return data

In [13]:
train_set = belirtilmemis_fonksiyonu(train_set)

In [14]:
def path_split_fonksiyonu(data):
    
    data = data.copy()
    data["il"], data["ilce"], data["mahalle"] = data["path"].str.split("/",2).str
    data["il"] = data["il"].astype(str).astype(int)
    data["ilce"] = data["ilce"].astype(str).astype(int)
    data["mahalle"] = data["mahalle"].astype(str).astype(int)
    
    return data

In [15]:
train_set = path_split_fonksiyonu(train_set)

C:\Users\YIGITC~1\AppData\Local\Temp/ipykernel_23216/3978551515.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  data["il"], data["ilce"], data["mahalle"] = data["path"].str.split("/",2).str


ValueError: invalid literal for int() with base 10: 'İstanbul'

In [16]:
def odasayisi_fonksiyonu(data):
   
    data = data.copy()
    data['odasayisi'], data['salonsayisi'] = data['odasayisi'].str.split("+",1).str
    data["odasayisi"] = data["odasayisi"].astype(str).astype(int)
    data["salonsayisi"] = data["salosayisi"].astype(str).astype(int)
    
    return data

In [17]:
train_set = odasayisi_fonksiyonu(train_set)

C:\Users\YIGITC~1\AppData\Local\Temp/ipykernel_23216/4152714293.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  data['odasayisi'], data['salonsayisi'] = data['odasayisi'].str.split("+",1).str


ValueError: invalid literal for int() with base 10: 'belirtilmemis'

In [18]:
label_encoder_cols = ("il","ilce","mahalle","emlaktipi","binayasi","bulundugukat","binadakikatsayisi","isitmatipi","otopark")

In [19]:
from sklearn.preprocessing import LabelEncoder
def label_encoder_fonksiyonu(data,cols):
    
    for col_name in label_encoder_cols:
        labelEncoder = LabelEncoder()
        labelEncoder.fit(data[col_name].values)
        data[col_name] = labelEncoder.transform(data[col_name].values)
        
    return data

In [20]:
train_set = label_encoder_fonksiyonu(train_set,label_encoder_cols)

KeyError: 'il'